## Load Data

In [65]:
function parseData(data::String)
    dataLines = split(data,"\n")
    lines = Array[]
    
    for line in dataLines
        splitLine = split(line,"")
        depth = 0
        expr = []
        for c in splitLine
            if c == "["
                depth += 1
            elseif c == "]"
                depth -= 1
            elseif c == ","
                #do nothing
            else
                push!(expr,[parse(Int,c),depth])
            end
        end
        push!(lines,expr)
    end
    return lines
end

parseData (generic function with 1 method)

In [154]:
testData1 = "[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]";

testData2 = "[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]";

In [155]:
test1 = parseData(testData1);
test2 = parseData(testData2);

In [219]:
rawData = open("snailHomework.txt") do file
    (read(file,String))
end
homeworkData = parseData(rawData);

## Part 1

In [210]:
function explodeNumber(num)
    indx = findfirst(x->x[2]>=5,num)
    if indx == 1 #exploding first pair
        num[3][1] += num[2][1] #add right number to next right
        num[2] = [0,num[1][2]-1] #replace exploded pair with single 0
        popfirst!(num)
        
    elseif indx == length(num) -1 #exploding last pair
        num[indx-1][1] += num[indx][1] #add left number to next left
        num[indx] = [0,num[indx][2]-1] #replace exploded pair with single 0
        pop!(num)
    else
        num[indx-1][1] += num[indx][1]
        num[indx+2][1] += num[indx+1][1]
        num[indx] = [0,num[indx][2]-1]
        deleteat!(num,indx+1)
    end
end

function splitNumber(num)
    indx = findfirst(x->x[1]>=10,num)
    dummy = num[indx]
    num[indx] = [Int(floor(dummy[1]/2)),dummy[2]+1]
    insert!(num,indx+1,[Int(ceil(dummy[1]/2)),dummy[2]+1])
end

function addNumbers(num1, num2)
    for i in 1:length(num1)
        num1[i][2] +=1
    end
    for i in 1:length(num2)
        num2[i][2] +=1
    end
    numsum = vcat(num1,num2)
    
    #reduce numsum
    toexplode = findfirst(x->x[2]>=5,numsum)
    tosplit = findfirst(x->x[1]>=10,numsum)
    #println(numsum)
    while !(isnothing(toexplode)) || !(isnothing(tosplit))
        if !(isnothing(toexplode))
            explodeNumber(numsum)
        else
            splitNumber(numsum)
        end
        #println(numsum)
        toexplode = findfirst(x->x[2]>=5,numsum)
        tosplit = findfirst(x->x[1]>=10,numsum)
    end
    return numsum
end

function magnitudeNumber(num)
    maxdepth = maximum([x[2] for x in num])
    #println(maxdepth)
    for depth in maxdepth:-1:1
        indxs = findall(x->x[2]==depth,num)
        #println(num)
        #println(indxs)
        for i in (length(indxs)-1):-2:1 #reverse iterate through list so we can delete in place
            #replace first element in pair with magnitude and reduce its depth, and delete second element
            num[indxs[i]] = [3*num[indxs[i]][1] + 2*num[indxs[i]+1][1], num[indxs[i]][2]-1]
            deleteat!(num,indxs[i]+1)
        end
    end
    return num[1][1]
end

magnitudeNumber (generic function with 1 method)

In [211]:
sumTest2 = deepcopy(test2[1])
for i in 2:length(test2)
    sumTest2 = addNumbers(sumTest2,deepcopy(test2[i]))    
end
println(sumTest2)
magnitudeNumber(sumTest2)

Any[[6, 4], [6, 4], [7, 4], [6, 4], [7, 4], [7, 4], [7, 4], [0, 4], [7, 4], [7, 4], [7, 4], [7, 4], [7, 4], [8, 4], [9, 4], [9, 4]]


4140

In [216]:
sumHw = deepcopy(homeworkData[1])
for i in 2:length(homeworkData)
    sumHw = addNumbers(sumHw,deepcopy(homeworkData[i]))
end
println(sumHw)
magnitudeNumber(sumHw)

Any[[6, 4], [7, 4], [7, 4], [7, 4], [0, 4], [7, 4], [7, 4], [7, 4], [7, 4], [7, 4], [8, 4], [8, 4], [8, 4], [7, 4], [3, 3]]


3756

### Testing explosion

In [95]:
teste = parseData("[[[[[9,8],1],2],3],4]")[1]
println(teste)
explodeNumber(teste)
println(teste)

Any[[9, 5], [8, 5], [1, 4], [2, 3], [3, 2], [4, 1]]
Any[[0, 4], [9, 4], [2, 3], [3, 2], [4, 1]]


In [96]:
teste = parseData("[7,[6,[5,[4,[3,2]]]]]")[1]
println(teste)
explodeNumber(teste)
println(teste)

Any[[7, 1], [6, 2], [5, 3], [4, 4], [3, 5], [2, 5]]
Any[[7, 1], [6, 2], [5, 3], [7, 4], [0, 4]]


In [97]:
teste = parseData("[[6,[5,[4,[3,2]]]],1]")[1]
println(teste)
explodeNumber(teste)
println(teste)

Any[[6, 2], [5, 3], [4, 4], [3, 5], [2, 5], [1, 1]]
Any[[6, 2], [5, 3], [7, 4], [0, 4], [3, 1]]


### Testing addition

In [132]:
num1 = parseData("[[[[4,3],4],4],[7,[[8,4],9]]]")[1]
num2 = parseData("[1,1]")[1]
addNumbers(num1,num2)

Any[[4, 5], [3, 5], [4, 4], [4, 3], [7, 3], [8, 5], [4, 5], [9, 4], [1, 2], [1, 2]]
Any[[0, 4], [7, 4], [4, 3], [7, 3], [8, 5], [4, 5], [9, 4], [1, 2], [1, 2]]
Any[[0, 4], [7, 4], [4, 3], [15, 3], [0, 4], [13, 4], [1, 2], [1, 2]]
Any[[0, 4], [7, 4], [4, 3], [7, 4], [8, 4], [0, 4], [13, 4], [1, 2], [1, 2]]
Any[[0, 4], [7, 4], [4, 3], [7, 4], [8, 4], [0, 4], [6, 5], [7, 5], [1, 2], [1, 2]]
Any[[0, 4], [7, 4], [4, 3], [7, 4], [8, 4], [6, 4], [0, 4], [8, 2], [1, 2]]


9-element Array{Any,1}:
 [0, 4]
 [7, 4]
 [4, 3]
 [7, 4]
 [8, 4]
 [6, 4]
 [0, 4]
 [8, 2]
 [1, 2]

In [156]:
mysum = deepcopy(test1[1])
for i in 2:length(test1)
    mysum = addNumbers(mysum,deepcopy(test1[i]))
    println(mysum)
end

Any[[4, 4], [0, 4], [5, 4], [4, 4], [7, 4], [7, 4], [6, 4], [0, 4], [8, 3], [7, 4], [7, 4], [7, 4], [9, 4], [5, 4], [0, 4]]
Any[[6, 4], [7, 4], [6, 4], [7, 4], [7, 4], [7, 4], [0, 4], [7, 4], [8, 4], [7, 4], [7, 4], [7, 4], [8, 4], [8, 4], [8, 4], [0, 4]]
Any[[7, 4], [0, 4], [7, 4], [7, 4], [7, 4], [7, 4], [7, 4], [8, 4], [7, 4], [7, 4], [8, 4], [8, 4], [7, 4], [7, 4], [8, 4], [7, 4]]
Any[[7, 4], [7, 4], [7, 4], [8, 4], [9, 4], [5, 4], [8, 4], [7, 4], [6, 4], [8, 4], [0, 4], [8, 4], [9, 4], [9, 4], [9, 4], [0, 4]]
Any[[6, 4], [6, 4], [6, 4], [6, 4], [6, 4], [0, 4], [6, 4], [7, 4], [7, 4], [7, 4], [8, 4], [9, 4], [8, 3], [8, 4], [1, 4]]
Any[[6, 4], [6, 4], [7, 4], [7, 4], [0, 4], [7, 4], [7, 4], [7, 4], [5, 4], [5, 4], [5, 4], [6, 4], [9, 2]]
Any[[7, 4], [8, 4], [6, 4], [7, 4], [6, 4], [8, 4], [0, 4], [8, 4], [7, 4], [7, 4], [5, 4], [0, 4], [5, 4], [5, 4], [5, 4], [6, 4]]
Any[[7, 4], [7, 4], [7, 4], [7, 4], [8, 4], [7, 4], [8, 4], [7, 4], [7, 4], [0, 4], [7, 4], [7, 4], [9, 2]]
Any[[8, 

In [157]:
test1[10]

6-element Array{Any,1}:
 [4, 4]
 [2, 4]
 [2, 3]
 [6, 2]
 [8, 2]
 [7, 2]

### Testing magnitude

In [195]:
testm = parseData("[9,1]")[1]
magnitudeNumber(testm)

Any[[9, 1], [1, 1]]
[1, 2]


29

In [196]:
testm = parseData("[[1,2],[[3,4],5]]")[1]
println(testm)
magnitudeNumber(testm)

Any[[1, 2], [2, 2], [3, 3], [4, 3], [5, 2]]
Any[[1, 2], [2, 2], [3, 3], [4, 3], [5, 2]]
[3, 4]
Any[[1, 2], [2, 2], [17, 2], [5, 2]]
[1, 2, 3, 4]
Any[[7, 1], [61, 1]]
[1, 2]


143

In [198]:
testm = parseData("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]")[1]
println(testm)
magnitudeNumber(testm)

Any[[0, 4], [7, 4], [4, 3], [7, 4], [8, 4], [6, 4], [0, 4], [8, 2], [1, 2]]


1384

In [200]:
testm = parseData("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")[1]
println(testm)
magnitudeNumber(testm)

Any[[8, 4], [7, 4], [7, 4], [7, 4], [8, 4], [6, 4], [7, 4], [7, 4], [0, 4], [7, 4], [6, 4], [6, 4], [8, 3], [7, 3]]


3488

## Part2

In [217]:
maxMagnitude = 0
for i in 1:length(test2),j in 1:length(test2)
    if i == j
        continue
    end
    magn = magnitudeNumber(addNumbers(deepcopy(test2[i]),deepcopy(test2[j])))
    if magn > maxMagnitude
        maxMagnitude = magn
    end
end
maxMagnitude

3993

In [220]:
maxMagnitude = 0
for i in 1:length(homeworkData),j in 1:length(homeworkData)
    if i == j
        continue
    end
    magn = magnitudeNumber(addNumbers(deepcopy(homeworkData[i]),deepcopy(homeworkData[j])))
    if magn > maxMagnitude
        maxMagnitude = magn
    end
end
maxMagnitude

4585